In [3]:
from openai import OpenAI
import pandas as pd
#from transformers import pipeline
from dotenv import load_dotenv
import os
from random import randint
#from transformers import AutoModel,AutoTokenizer


In [ ]:
# 

def clean_sentence(sentence):
    # Remove quotation marks and similar characters
    cleaned = sentence.replace('"', '').replace("'", "")
    return cleaned.strip()

# Load API key from .env file
def get_credentials():
    load_dotenv()
    api_key = os.getenv('OPENAI_API_KEY')
    return api_key   

    
    
def generate_swedish_sentences(entity_list, entities_metadata, num_sentences, sentence_start):
    
    completion = client.chat.completions.create(
        temperature=1.0,
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Du är en hjälpsam assistent som genererar spontana, varierade, och autentiska svenska meningar."},

            {
                "role": "user",
                "content": (
                    f"Generera en kort, spontan och vardaglig text på svenska ( max {num_sentences} meningar) "
                    f"Meningen skall börja med {sentence_start}"
                    f"Meningarna skall innehåll följande entititer {entity_list}, vidare finns en beskrivning av varje entitet: {entities_metadata}."
                    "Använd entiteterna på ett naturligt sätt så som de faktiskt skulle användas i vanligt tal. "
                    "Undvik formella juridiska benämningar såsom 'med firma', 'filial', 'aktiebolaget' eller 'ideella föreningen' "
                    "om de inte är naturliga i vardagsspråk. Variera sammanhang, stilar och teman så att varje text känns unik. "
                    "Variera alltid meningarnas början, struktur och stil; använd frågor, kommentarer, utrop eller påståenden för variation. "
                    "Blanda in humor eller känslor för att texten ska kännas genuint vardaglig."
                    
                )
            }
        ]
    )

    raw_sentence = completion.choices[0].message.content
    cleaned_sentence = clean_sentence(raw_sentence)
    print(cleaned_sentence)
    return cleaned_sentence


client = OpenAI(api_key=get_credentials())

In [ ]:
entity_df = pd.read_csv('./entities/manual/entity_df.csv')
entity_map = {"organisationsnamn": 1, "postadress": 2}
num_samples = 10
company_names_to_ignore = ["Aktiebolaget", "Handelsbolaget", "Aktiebolag", "Handelsbolag"]

sentence_starters = [
    "Jag har en fråga om",
    "Jag undrar om",
    "Kan du berätta om",
    "Vet du något om",
    "Har du koll på",
    "Jag såg nyligen att",
    "Har du hört något om",
    "Vad tycker du om",
    "Fan vad trött man är på",
    "Det är riktigt nice med",
    "Problemed med det är att",
    "Jag läste att",
    "Vet du vart man kan hitta",
    "Jag funderar på",
    "Det vore kul att testa",
    "Ja, jag ska kolla upp",
    "Har du hört något nytt om",
    "Kan du rekommendera",
    "Jag skulle behöva tips på",
    "Tror du det går att fixa",
    "Vet du var man hittar",
    "Det verkar spännande med",
    "Jag är nyfiken på",
    "Är du insatt i",
    "Hur är det med",
    "Har du erfarenhet av",
    "Jag läste precis om",
    "Har du varit på",
    "Jag funderar på att prova",
    "Man borde egentligen testa",
    "Känner du någon som",
    "En polare till mig var",
    "En kille jag känner sa att",
    "En tjejkompis till mig var så less på",
    "Jag fattar inte hur man använder",
    "Vad är grejen med"
    "Vad har vi på",
    "Jag är sjukt taggad på",
    "Kan du berätta lite om",
    "Det vore intressant att höra",
    "Jag behöver veta mer om",
    "Jag fattar inte hypen med",
    "Snälla hjälp mig förstå",
    "Finns det någon särskild anledning att",
    "Vad hände ens med",
    "Kan man använda",
    "Jag är lite osäker på",
    "Jag skulle vilja veta mer om",
    "Har du någonsin använt",
    "Vad är din åsikt om",
    "Vad tycker du om",
    "Tror du att man kan lita på",
    "Kan man lita på"
]

In [ ]:


sentences = []
for i in range(num_samples):
    entity_list = []
    entities_metadata = []
    for j in range(2):
        entity_type_picker = 0 if randint(1, 100) <= 95 else 1
        entity_picker = randint(0, len(entity_df) - 1)
        entity = entity_df.iloc[entity_picker]
        entity_type = list(entity_map.keys())[entity_type_picker]
        entity_name = entity[entity_type]

        # Remove company prefixes
        for ignore_name in company_names_to_ignore:
            entity_name = entity_name.replace(ignore_name, "")
        
        entity_list.append(entity_name)
    
        if entity_type == "organisationsnamn":
            entities_metadata.append({"entity_name": entity_name, "entity_description": entity["verksamhetsbeskrivning"], "organisation_number": entity["organisationsidentitet"]})
        elif entity_type == "postadress":
            entities_metadata.append({"entity_name": entity_name, "entity_description": "Adress"})
        
    print(entity_list)
    print(entities_metadata)    
    
    sentence_start = sentence_starters[randint(0, len(sentence_starters) - 1)]
    num_sentences = randint(1, 3)
    sentence = generate_swedish_sentences(entity_list, entities_metadata, num_sentences, sentence_start)
    sentences.append({"sentence": sentence, "entities": entity_list, "metedata": entities_metadata})
    

    

    

['"Roger Frengen Bygg Ltd UK', '"Box 417']
[{'entity_name': '"Roger Frengen Bygg Ltd UK', 'entity_description': '"Filialen ska bedriva byggverksamhet."', 'organisation_number': '5164046905'}, {'entity_name': '"Box 417', 'entity_description': 'Adress'}]
Hörde du att Roger Frengen Bygg Ltd UK har flyttat sitt kontor till Box 417? Det är ju typiskt när man trott att man hittat den perfekta byggfirman, då byter de adress! Jag hoppas de har bättre skilsmässa med posten än vad jag har!
['"Byggaktiebolaget', '"ZyXEL Communications A/S, Danmark,']
[{'entity_name': '"Byggaktiebolaget', 'entity_description': '"Bolaget skall förvärva, bebygga, förvalta och försälja fastigheter, idka byggnadsverksamhet, förvärv och förvalta lös egendom samt idka därmed förenlig verksamhet."', 'organisation_number': '5560560558'}, {'entity_name': '"ZyXEL Communications A/S, Danmark,', 'entity_description': '"Filialen skall bedriva försäljning av hårdvaruutrustning och därmed förenlig verksamhet inom Sverige."', 'or

In [52]:
for i in sentences: print(i["sentence"])

Hörde du att Roger Frengen Bygg Ltd UK har flyttat sitt kontor till Box 417? Det är ju typiskt när man trott att man hittat den perfekta byggfirman, då byter de adress! Jag hoppas de har bättre skilsmässa med posten än vad jag har!
Visste du att Byggaktiebolaget just har fått ett stort uppdrag i stan? Det blir spännande att se hur de kommer att förvandla området. Förresten, jag hörde att ZyXEL Communications A/S, Danmark, också planerar att öka sin försäljning här med ny hårdvaruteknik – framtiden ser lovande ut!
Har du hört om Umeå pool? De har verkligen ett brett utbud av pooler som ser fantastiska ut, perfekt för sommaren! Å andra sidan, F Hemberg Invest sysslar mer med förvaltning av aktier och värdepapper – kanske något för de som hellre håller sig inomhus med sina investeringar?
I dag gick jag förbi Marie’s Salladsbar och fick plötsligt cravings för en fräsch kebabsallad – de har ju helt oslagbara såser! Kanske efter jag har handlat på Fairfox Enterprise för att hitta ett par sny

In [54]:
sentences_df = pd.DataFrame(sentences)  
sentences_df.to_csv('./sentences_df.csv', index=False)